In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/GitHub/ERA-V1/Assignments/S9

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/GitHub/ERA-V1/Assignments/S9


In [2]:
%%writefile src/data_setup.py
import os
import numpy as np

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset


NUM_WORKERS = os.cpu_count()

class AlbumentationsDataset(Dataset):
    def __init__(self, dataset, transforms):
        self.dataset = dataset
        self.transforms = transforms
        self.classes = dataset.classes

    def __getitem__(self, index):
        image, target = self.dataset[index]

        image = np.array(image)

        # Apply Albumentations transforms
        transformed = self.transforms(image=image)
        image = transformed['image']

        return image, target

    def __len__(self):
        return len(self.dataset)

def create_dataloaders(train_dir, test_dir, train_transforms, test_transforms, batch_size, num_workers=NUM_WORKERS):

  # Download the dataset
  train_data = datasets.CIFAR10(train_dir, train=True, download=True)
  test_data = datasets.CIFAR10(test_dir, train=False, download=True)

  # Wrap the datasets with AlbumentationsDataset
  train_data = AlbumentationsDataset(train_data, train_transforms)
  test_data = AlbumentationsDataset(test_data, test_transforms)

  # Get class names
  class_names = train_data.classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names

Overwriting src/data_setup.py


In [3]:
%%writefile src/model_builder.py
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model1(nn.Module):
    def __init__(self, dropout_value=0.01):
        super(Model1, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=3, stride=1, bias=False, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(dropout_value),
            nn.Conv2d(8, 8, kernel_size=3, stride=1, bias=False, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(dropout_value),
            nn.Conv2d(8, 8, kernel_size=3, stride=2, bias=False, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(dropout_value))
        self.block2 = nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=3, stride=1, bias=False, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value),
            nn.Conv2d(16, 16, kernel_size=3, stride=1, bias=False, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value),
            nn.Conv2d(16, 16, kernel_size=3, stride=2, bias=False, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(dropout_value))
        self.block3 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, bias=False, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(dropout_value),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, bias=False, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(dropout_value),
            nn.Conv2d(32, 32, kernel_size=3, stride=2, bias=False, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(dropout_value))
        self.block4 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, bias=False, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(dropout_value),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, bias=False, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(dropout_value),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, bias=False, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(dropout_value))
        self.gap = nn.Sequential(
            nn.AdaptiveAvgPool2d(1))
        self.block5 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=1, stride=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Dropout(dropout_value),
            nn.Conv2d(128, 10, kernel_size=1, stride=1, bias=False))

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.gap(x)
        x = self.block5(x)
        x = x.view((x.shape[0],-1))
        x = F.log_softmax(x, dim=1)
        return x


Overwriting src/model_builder.py


In [4]:
!pip install torchsummary
import torch
from src.model_builder import Model1 as Net
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(3, 32, 32))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 32, 32]             216
              ReLU-2            [-1, 8, 32, 32]               0
       BatchNorm2d-3            [-1, 8, 32, 32]              16
           Dropout-4            [-1, 8, 32, 32]               0
            Conv2d-5            [-1, 8, 32, 32]             576
              ReLU-6            [-1, 8, 32, 32]               0
       BatchNorm2d-7            [-1, 8, 32, 32]              16
           Dropout-8            [-1, 8, 32, 32]               0
            Conv2d-9            [-1, 8, 16, 16]             576
             ReLU-10            [-1, 8, 16, 16]               0
      BatchNorm2d-11            [-1, 8, 16, 16]              16
          Dropout-12            [-1, 8, 16, 16]               0

In [5]:
%%writefile src/engine.py
from src.utils import plot_graph, show_incorrect_images
import torch

from tqdm.auto import tqdm

def GetCorrectPredCount(pPrediction, pLabels):
  return pPrediction.argmax(dim=1).eq(pLabels).sum().item()

def train_step(model, device, train_loader, optimizer, criterion):
  model.train()
  pbar = tqdm(train_loader)

  train_loss = 0
  correct = 0
  processed = 0

  for batch_idx, (data, target) in enumerate(pbar):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()

    # Predict
    pred = model(data)

    # Calculate loss
    loss = criterion(pred, target)
    train_loss+=loss.item()

    # Backpropagation
    loss.backward()
    optimizer.step()

    correct += GetCorrectPredCount(pred, target)
    processed += len(data)

    pbar.set_description(desc= f'Train: Loss={loss.item():0.4f} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')

  train_acc = 100*correct/processed
  return train_loss, train_acc

def test_step(model, device, test_loader, criterion):
    model.eval()

    test_loss = 0
    correct = 0
    test_incorrect_pred = {'images': [], 'ground_truths': [], 'predicted_vals': []}

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)

            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss

            pred = output.argmax(dim=1)
            correct_mask = pred.eq(target)
            incorrect_indices = ~correct_mask

            test_incorrect_pred['images'].extend(data[incorrect_indices])
            test_incorrect_pred['ground_truths'].extend(target[incorrect_indices])
            test_incorrect_pred['predicted_vals'].extend(pred[incorrect_indices])

            correct += GetCorrectPredCount(output, target)


    test_loss /= len(test_loader.dataset)
    test_acc = 100. * correct / len(test_loader.dataset)


    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss, test_acc, test_incorrect_pred

def train(model, train_loader, test_loader, device, optimizer, epochs, criterion):

    class_map = {
        0: 'airplane',
        1: 'automobile',
        2: 'bird',
        3: 'cat',
        4: 'deer',
        5: 'dog',
        6: 'frog',
        7: 'horse',
        8: 'ship',
        9: 'truck'
    }
    # Data to plot accuracy and loss graphs
    # Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }


    for epoch in range(epochs):
        print(f'Epoch {epoch}')
        train_loss, train_acc = train_step(model=model, device=device, train_loader=train_loader, optimizer=optimizer, criterion=criterion)
        test_loss, test_acc, test_incorrect_pred = test_step(model=model, device=device, test_loader=test_loader, criterion=criterion)

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    plot_graph(results["train_loss"], results["test_loss"], results["train_acc"], results["test_acc"])
    show_incorrect_images(test_incorrect_pred, class_map)

    # Return the filled results at the end of the epochs
    return results

Overwriting src/engine.py


In [6]:
%%writefile src/utils.py
import torch
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

def save_model(model, target_dir, model_name):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

def plot_graph(train_losses, test_losses, train_acc, test_acc):
    fig, axs = plt.subplots(1, 2, figsize=(15, 6))

    # Plot Train and Test Loss
    axs[0].plot(train_losses, label='Train Loss')
    axs[0].plot(test_losses, label='Test Loss')
    axs[0].set_title("Train and Test Loss")
    axs[0].set_xlabel("Epoch")
    axs[0].set_ylabel("Loss")
    axs[0].legend()

    # Plot Train and Test Accuracy
    axs[1].plot(train_acc, label='Train Accuracy')
    axs[1].plot(test_acc, label='Test Accuracy')
    axs[1].set_title("Train and Test Accuracy")
    axs[1].set_xlabel("Epoch")
    axs[1].set_ylabel("Accuracy")
    axs[1].legend()

    plt.savefig("models/loss_accuracy_plot.png")

def show_incorrect_images(test_incorrect_pred, class_map):
    num_images = 10
    num_rows = 2
    num_cols = (num_images + 1) // 2  # Adjust the number of columns based on the number of images

    fig, axs = plt.subplots(num_rows, num_cols, figsize=(num_cols * 2, num_rows * 2))

    for i in range(num_images):
        row_idx = i // num_cols
        col_idx = i % num_cols

        img = test_incorrect_pred['images'][i].cpu().numpy()
        img = np.transpose(img, (1, 2, 0))
        img = (img - np.min(img)) / (np.max(img) - np.min(img))  # Normalize the image data
        label = test_incorrect_pred['ground_truths'][i].cpu().item()
        pred = test_incorrect_pred['predicted_vals'][i].cpu().item()

        axs[row_idx, col_idx].imshow(img)
        axs[row_idx, col_idx].set_title(f'GT: {class_map[label]}, Pred: {class_map[pred]}')
        axs[row_idx, col_idx].axis('off')

    plt.savefig("models/incorrect_images.png")

Overwriting src/utils.py


In [7]:
%%writefile train.py
import os
import torch
from src import data_setup, engine, model_builder, utils
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torchvision import transforms

# Setup hyperparameters
NUM_EPOCHS = 20
BATCH_SIZE = 128
LEARNING_RATE = 0.001
MOMENTUM = 0.9

# Setup directories
train_dir = "../data"
test_dir = "../data"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
# Train Phase transformations
train_transforms = A.Compose([
    A.HorizontalFlip(p=1),  # Apply horizontal flip with probability 1 (always)
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=10, p=0.5),  # Apply shift, scale, and rotation
    A.CoarseDropout(max_holes=1, max_height=16, max_width=16, min_holes=1, min_height=16, min_width=16, fill_value=(0.49139968, 0.48215827, 0.44653124), mask_fill_value=None),  # Apply coarse dropout
    A.Normalize(mean=[0.49139968, 0.48215827, 0.44653124], std=[0.24703233, 0.24348505, 0.26158768]),  # Normalize the image
    ToTensorV2() # Convert image to a PyTorch tensor
])

#train_transforms = transforms.Compose([
#                                       transforms.ToTensor(),
#                                       transforms.Normalize((0.49139968, 0.48215827, 0.44653124), (0.24703233, 0.24348505, 0.26158768))
#                                       ])

# Test Phase transformations
test_transforms = A.Compose([
    A.Normalize(mean=[0.49139968, 0.48215827, 0.44653124], std=[0.24703233, 0.24348505, 0.26158768]),  # Normalize the image
    ToTensorV2()  # Convert image to a PyTorch tensor
])

#test_transforms = transforms.Compose([
#                                       transforms.ToTensor(),
#                                       transforms.Normalize((0.49139968, 0.48215827, 0.44653124), (0.24703233, 0.24348505, 0.26158768))
#                                       ])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_builder.Model1().to(device)

# Set loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)


# Start training with help from engine.py
engine.train(model=model,
             train_loader=train_dataloader,
             test_loader=test_dataloader,
             criterion=criterion,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")

Overwriting train.py


In [8]:
!pip install albumentations
!python train.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Files already downloaded and verified
Files already downloaded and verified
Epoch 0
Train: Loss=1.6289 Batch_id=390 Accuracy=27.49: 100% 391/391 [00:19<00:00, 19.83it/s]
Test set: Average loss: 0.0134, Accuracy: 3763/10000 (37.63%)

Epoch 1
Train: Loss=1.7012 Batch_id=390 Accuracy=36.39: 100% 391/391 [00:17<00:00, 22.85it/s]
Test set: Average loss: 0.0125, Accuracy: 4164/10000 (41.64%)

Epoch 2
Train: Loss=1.4996 Batch_id=390 Accuracy=39.59: 100% 391/391 [00:19<00:00, 19.78it/s]
Test set: Average loss: 0.0117, Accuracy: 4581/10000 (45.81%)

Epoch 3
Train: Loss=1.4881 Batch_id=390 Accuracy=42.67: 100% 391/391 [00:18<00:00, 21.08it/s]
Test set: Average loss: 0.0113, Accuracy: 4766/10000 (47.66%)

Epoch 4
Train: Loss=1.4197 Batch_id=390 Accuracy=44.50: 100% 391/391 [00:18<00:00, 21.14it/s]
Test set: Average loss: 0.0109, Accuracy: 5013/10000 (50.13%)

Epoch 5
Train: Loss=1.5183 Batch_id=390 

In [9]:

!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   S9.ipynb
	modified:   src/__pycache__/data_setup.cpython-310.pyc
	modified:   src/__pycache__/engine.cpython-310.pyc
	modified:   src/__pycache__/model_builder.cpython-310.pyc
	modified:   src/__pycache__/utils.cpython-310.pyc
	modified:   src/data_setup.py
	modified:   src/engine.py
	modified:   src/model_builder.py
	modified:   train.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../S5/__pycache__/
	../S7/__pycache__/
	../S7/data/
	models/
	../data/

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add S9.ipynb